In [ ]:
# Monta Google Drive per accedere ai file
from google.colab import drive
drive.mount('/content/drive')

# Importa le librerie necessarie
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense

# Carica il dataset da Google Drive
file_path = "/content/drive/MyDrive/cyber/definitivo.csv"
df = pd.read_csv(file_path)

# Rimuovi spazi bianchi dai nomi delle colonne
df.columns = df.columns.str.strip()

# Converti la colonna 'Normal/Attack' in numeri
df['Normal/Attack'] = df['Normal/Attack'].map({'Normal': 0, 'Attack': 1})

# Rimuovi la colonna 'Timestamp' poiché non è necessaria per il rilevamento delle anomalie
df = df.drop(columns=['Timestamp'])

# Rimuovi tutte le righe che contengono almeno un valore NaN
df.dropna(inplace=True)

# Dividi il dataset in features (X) e target (y)
X = df.drop(columns=['Normal/Attack'])
y = df['Normal/Attack']

# Standardizza le features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividi il dataset in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Reshape dei dati di input per includere le informazioni temporali
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Crea il modello
def create_lstm_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
# Imposta la forma dei dati correttamente per l'input del modello LSTM
input_shape = (X_train.shape[1], 1)

# Crea il modello
model = create_lstm_model(input_shape)

# Fai il fit del modello
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Valuta il modello sul set di test
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Effettua previsioni sul set di test
y_pred = model.predict(X_test)

# Imposta una soglia per classificare le previsioni come anomalie
threshold = 0.5
# Confronta le previsioni con la soglia per identificare gli esempi anomali
y_pred_anomalies = (y_pred > threshold).astype(int)


# Analyzing the model




In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, average_precision_score

## Classification Report


**Precision**: Represents the proportion of instances classified as positive that are actually positive. In other words, it is the model's ability to not mislabel negative instances as positive. For class 0 (non-anomaly), precision is 100%, meaning all instances classified as non-anomalies are actually non-anomalies. For class 1 (anomaly), precision is 98%, indicating that 2% of instances classified as anomalies might be false positives.

**Recall**: Represents the proportion of total positive instances that were correctly identified by the model. In other words, it is the model's ability to find all positive instances. For class 0, recall is 100%, indicating that all non-anomaly instances were correctly identified. For class 1, recall is 95%, meaning that 5% of anomaly instances might not have been detected by the model.

**F1-score**: It is a weighted average of precision and recall. It represents the balance between precision and recall. A high F1-score indicates a good balance between precision and recall.

**Support**: Represents the number of instances in each class in the test dataset.

**Accuracy**: Represents the proportion of correct predictions out of the total predictions made by the model. In your case, accuracy is 100%, meaning all predictions made by the model are correct in the test dataset.

In [ ]:
# Valuta le prestazioni del modello
print(classification_report(y_test, y_pred_anomalies))

## Learning Curves
The learning curve can provide valuable insights into the performance of a machine learning model during training.
As we can see from these two learning curves that analyze both loss and accuracy, the model seems to be learning effectively and generalizing well to unseen data.

* Both the training and validation loss curves show a clear downward trend, indicating that the model is consistently improving its ability to correctly classify instances.
* The gap between the training and validation curves remains small throughout the training process, suggesting that the model is not significantly overfitting the training data. This is crucial for good generalization to unseen data.
* Both training and validation accuracy curves show an upward trend, reaching high final values. This implies that the model is correctly classifying both training and unseen data with a high degree of accuracy.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Learning Curve: Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Learning Curve: Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


The confusion matrix helps visualize true positives, true negatives, false positives, and false negatives.

As we already discussed for the curves, this confusion matrix further strengthens the case that the model is a highly effective anomaly detector.

* Just like the low training and validation loss in the curves, the TPR of 0.98 in the confusion matrix confirms the model's ability to correctly identify real attacks. This signifies minimal instances slipping through the cracks.
* The TNR of 1.00 echoes the small gap between curves, indicating excellent generalization. The model rarely misclassifies normal instances as attacks, minimizing false alarms.
* The precision of 0.98 for attacks reiterates the curves' suggestion of minimal overfitting. The model focuses on true positives, reducing noise and false positives.
* The recall of 0.98 for attacks aligns with the high validation accuracy, reflecting the model's ability to capture most attacks. Missed attacks are rare.
* With only 387 false positives, the confusion matrix emphasizes the curves' indication of minimal overfitting.

In [ ]:
cm = confusion_matrix(y_test, y_pred_anomalies)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()




The ROC curve illustrates the trade-off between sensitivity and specificity at various threshold settings. Based on the combination of all the analyses, it's likely that your model slightly prioritizes sensitivity, especially at stricter thresholds. However, it maintains a good overall balance between sensitivity and specificity across various thresholds.



In [ ]:
# Plotting ROC curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

The precision-recall curve illustrates the trade-off between precision and recall. As we can see the curve starts very close to the top (representing perfect precision) and gradually decreases as recall increases. This means the model can achieve very high precision even when only a small portion of actual attacks are identified.

The decrease in precision as recall increases is relatively slow, indicating that the model maintains good precision even as it identifies more attacks. This is a positive sign, as it suggests the model does not generate a large number of false positives when trying to catch more attacks.





In [ ]:
# Plotting Precision-Recall curve

precision, recall, _ = precision_recall_curve(y_test, y_pred)
average_precision = average_precision_score(y_test, y_pred)

plt.figure(figsize=(8, 6))
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
plt.grid(True)
plt.show()


The histogram of predicted anomaly scores shows the distribution of anomaly scores across the test set. Basing on this histogram it seems like this model's predictions are indeed slightly skewed towards the attack class (higher values), consistent with what I mentioned earlier. Here's a more detailed analysis:

**Observations**:

* **Shape**: The histogram has a relatively smooth, unimodal distribution, as expected for a well-performing model.
* **Distribution**: The peak of the distribution leans slightly towards the right side, indicating that a larger portion of predictions fall into higher values, which represent predicted attacks. This aligns with the idea that the model prioritizes sensitivity to some extent, aiming to catch most attacks even if it means some false positives.
* **Outliers**: There are a few outliers on both ends of the histogram, but they are relatively infrequent. This suggests that most predictions are confident (closer to either 0 or 1), while a small number fall into the less confident range.

In [ ]:
# Plotting Histogram of Predicted Anomaly Scores
plt.figure(figsize=(8, 6))
plt.hist(y_pred, bins=50, color='blue', alpha=0.7)
plt.xlabel('Predicted Anomaly Score')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Anomaly Scores')
plt.grid(True)
plt.show()

# Attack model


It's important to clarify that the DAICS paper itself doesn't propose attacks against the anomaly detection model. Its focus is on improving the detection of actual anomalies, not exploring ways to bypass it.

However, the field of anomaly detection does consider potential attacks and vulnerabilities, so it's valuable to discuss them even if not directly addressed in the specific paper. Here are some general attack types relevant to anomaly detection models like DAICS:

Adversarial attacks: These involve crafting specific inputs that intentionally trigger false positives or negatives. In the context of ICSs, this could involve manipulating sensor data to appear normal while hiding malicious activity.

Data poisoning: Attackers inject malicious data into the training dataset to manipulate the model's behavior. This could be difficult for DAICS since it requires few data samples, but not impossible.

Model inversion: Attackers try to recover sensitive information from the model itself, potentially revealing normal behavior patterns and making it easier to craft attacks.

Evasion attacks: Attackers modify their actions to avoid detection by the model while still achieving their goals. This could involve mimicking normal behavior or exploiting specific weaknesses in the model's decision-making process.

Synergy attacks: Attackers combine multiple techniques, like data poisoning and evasion, to make their attacks more effective.

It's important to note that DAICS does incorporate some measures to mitigate these vulnerabilities, such as its focus on learning changes in behavior and its robustness to noise. However, no model is completely immune to attack, and ongoing research is crucial to staying ahead of potential threats.

If you're interested in learning more about specific attacks against anomaly detection models or the defense mechanisms employed by DAICS, feel free to ask!

In [ ]:
import numpy as np

# Generate synthetic attacks by adding outliers to the test set
def add_outliers(X_test, outlier_ratio=1):
    num_outliers = int(len(X_test) * outlier_ratio)
    indices = np.random.choice(len(X_test), num_outliers, replace=False)
    X_test_outliers = X_test.copy()
    for idx in indices:
        # Add outliers by multiplying random noise to a random feature
        feature_idx = np.random.randint(0, X_test.shape[1])
        X_test_outliers[idx, feature_idx] *= np.random.uniform(5, 10)  # Add outlier by scaling
    return X_test_outliers

# Simulate attacks by adding outliers to the test set
X_test_attacked = add_outliers(X_test)

# Evaluate the model on attacked test set
test_loss_attacked, test_accuracy_attacked = model.evaluate(X_test_attacked, y_test)
print("Test Accuracy on Attacked Test Set:", test_accuracy_attacked)


In [ ]:
# Generate synthetic attacks by adding malicious instances to the training set
def add_malicious_instances(X_train, y_train, malicious_ratio=1):
    num_malicious = int(len(X_train) * malicious_ratio)
    malicious_instances = np.random.rand(num_malicious, X_train.shape[1])  # Generate random malicious instances
    malicious_labels = np.ones(num_malicious)  # Assign malicious labels
    X_train_malicious = np.concatenate([X_train, malicious_instances])
    y_train_malicious = np.concatenate([y_train, malicious_labels])
    return X_train_malicious, y_train_malicious

# Simulate a data poisoning attack by adding malicious instances to the training set
X_train_poisoned, y_train_poisoned = add_malicious_instances(X_train, y_train)

# Train the model on the poisoned training set
poisoned_model = create_lstm_model(input_shape)
history_poisoned = poisoned_model.fit(X_train_poisoned, y_train_poisoned, epochs=1, batch_size=128, validation_split=0.2)

# Evaluate the poisoned model on the original test set
test_loss_poisoned, test_accuracy_poisoned = poisoned_model.evaluate(X_test, y_test)
print("Test Accuracy of Poisoned Model on Original Test Set:", test_accuracy_poisoned)


In [ ]:
import numpy as np

# Function to generate adversarial examples using FGSM with NumPy
def fgsm_attack_numpy(model, X, y, epsilon=0.5):
    # Compute gradients of the loss with respect to the input data
    gradient = compute_gradient(model, X, y)

    # Compute the sign of the gradient
    sign_gradient = np.sign(gradient)

    # Generate perturbation
    perturbation = epsilon * sign_gradient

    # Create adversarial examples
    adversarial_X = X + perturbation.T  # Transpose perturbation to match the shape of X

    return adversarial_X

# Function to compute gradient of the loss with respect to the input data
def compute_gradient(model, X, y):
    # Forward pass
    logits = model.predict(X)

    # Convert y to NumPy array and reshape it
    y_array = np.array(y).reshape(-1, 1)

    # Compute loss gradient manually (assuming binary cross-entropy loss)
    error = logits - y_array
    gradient = X.T @ error / len(X)

    return gradient

# Generate adversarial examples using FGSM with NumPy
epsilon = 0.1  # Magnitude of perturbation
X_test_adv_numpy = fgsm_attack_numpy(model, X_test, np.array(y_test), epsilon)

# Evaluate the model on the adversarial examples
test_loss_adv_numpy, test_accuracy_adv_numpy = model.evaluate(X_test_adv_numpy, y_test)
print("Test Accuracy on Adversarial Examples (NumPy):", test_accuracy_adv_numpy)


In [ ]:
import numpy as np

# Importa le librerie necessarie
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Crea il modello autoencoder LSTM
class LSTM_Autoencoder:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.autoencoder = self.build_autoencoder()

    def build_encoder(self):
        encoder = Sequential([
            LSTM(32, input_shape=self.input_shape, return_sequences=True),
            LSTM(16, return_sequences=False)
        ])
        return encoder

    def build_decoder(self):
        decoder = Sequential([
            RepeatVector(self.input_shape[0]),
            LSTM(16, return_sequences=True),
            LSTM(32, return_sequences=True),
            TimeDistributed(Dense(self.input_shape[1]))
        ])
        return decoder

    def build_autoencoder(self):
        autoencoder = Sequential([self.encoder, self.decoder])
        autoencoder.compile(optimizer='adam', loss='mse')
        return autoencoder

    def train(self, X_train, epochs=10, batch_size=128, validation_split=0.2):
        history = self.autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def encode(self, X):
        encoded_data = self.encoder.predict(X)
        return encoded_data

    def decode(self, encoded_data):
        decoded_data = self.decoder.predict(encoded_data)
        return decoded_data

# Imposta la forma dei dati correttamente per l'input dell'autoencoder LSTM
input_shape_autoencoder = (X_train.shape[1], 1)

# Crea il modello autoencoder
autoencoder_model = LSTM_Autoencoder(input_shape_autoencoder)

# Fai il fit dell'autoencoder
autoencoder_history = autoencoder_model.train(X_train_reshaped)

# Usa l'autoencoder per ottenere le rappresentazioni latenti
encoded_train = autoencoder_model.encode(X_train_reshaped)
encoded_test = autoencoder_model.encode(X_test_reshaped)

# Successivamente, potresti utilizzare le rappresentazioni latenti come input per un'altra rete neurale per la classificazione o il rilevamento di anomalie.
